---
title: "Practice Activity 6.1 — Polynomial_Polynomial_Regression"
format:
  html:
    embed-resources: true   
    toc: true
    toc-depth: 2
execute:
  echo: true
  warning: false
  error: false
  freeze: auto              
  cache: true
---

[Practice Activity 6.1](https://github.com/shiqiwu212/GSB-S544-01/tree/7775414dd89ce55b2443e5f4eb96d0eb84f9fdfe/Week%206/Practice%20Activities/Practice%20Activity%206.1)


# Palmer Penguins Modeling

Import the Palmer Penguins dataset and print out the first few rows.

Suppose we want to predict `bill_depth_mm` using the other variables in the dataset.

Which variables would we need to **dummify**?

In [ ]:
#Import the Palmer penguins dataset and print out he first few rows.
import pandas as pd
from palmerpenguins import load_penguins
from plotnine import ggplot, aes, geom_point, geom_boxplot, geom_bar, geom_histogram, geom_density, facet_wrap
penguins = load_penguins()
penguins 

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
...,...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009


In [8]:
# Suppose we want to predict `bill_depth_mm` using the other variables in the dataset. According to 10.4.1: Target and Predictors.
y = penguins['bill_depth_mm']
X = penguins[['bill_length_mm', 'flipper_length_mm', 'body_mass_g','species', 'island', 'sex', 'year']]

Which variables would we need to **dummify**?

Answer: The variables we need to dummify should be sex, year, island. 

Let's use `bill_length_mm` to predict `bill_depth_mm`. Prepare your data and fit the following models on the entire dataset:

* Simple linear regression (e.g. straight-line) model
* Quadratic (degree 2 polynomial) model
* Cubic (degree 3 polynomial) model
* Degree 10 polynomial model

Make predictions for each model and plot your fitted models on the scatterplot.

In [ ]:
# Set Up
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
# 11.5.1 Dummifying Categorical Variables
X_enc = pd.get_dummies(X, drop_first=True)

# 11.4.3: Variable prep happens before any model fit
valid = y.notna() & X_enc.notna().all(axis=1)
X_enc = X_enc.loc[valid]
y_clean = y.loc[valid]

# 10.4.3 Test/training split
X_train, X_test, y_train, y_test = train_test_split(X_enc, y_clean, test_size=0.25)

# 10.4.2 Model specifications
knn = KNeighborsRegressor()
lr  = LinearRegression()
dt  = DecisionTreeRegressor()

# 10.4.4 Model fitting
lr_fit  = lr.fit(X_train, y_train)
dt_fit  = dt.fit(X_train, y_train)
knn_fit = knn.fit(X_train, y_train)

# 10.4.4 Predictions
y_pred_knn = knn_fit.predict(X_test)
y_pred_lr  = lr_fit.predict(X_test)
y_pred_dt  = dt_fit.predict(X_test)

# 10.4.4 Metrics and model comparison
mse_knn = mean_squared_error(y_test, y_pred_knn)
mse_lr  = mean_squared_error(y_test, y_pred_lr)
mse_dt  = mean_squared_error(y_test, y_pred_dt)
print(mse_knn, mse_lr, mse_dt)

# 10.4.4 Final model step
final_model = lr.fit(X_enc, y_clean)
final_model.coef_

2.1429255813953496 0.8133247472425623 1.2144186046511627


array([ 4.38012719e-02,  2.69744026e-02,  4.76697435e-04, -1.56268696e-01,
       -4.73967701e-01, -5.18568821e+00, -1.34379860e-01,  6.48458729e-03,
        8.19378886e-01])

In [15]:
# 10.4.4 Training-set predictions 
y_tr_pred_lr  = lr_fit.predict(X_train)
y_tr_pred_dt  = dt_fit.predict(X_train)
y_tr_pred_knn = knn_fit.predict(X_train)

# 10.4.4 Training MSE 
mse_tr_lr  = mean_squared_error(y_train, y_tr_pred_lr)
mse_tr_dt  = mean_squared_error(y_train, y_tr_pred_dt)
mse_tr_knn = mean_squared_error(y_train, y_tr_pred_knn)

print("TRAIN MSEs:", {"LR": mse_tr_lr, "DT": mse_tr_dt, "KNN": mse_tr_knn})

TRAIN MSEs: {'LR': 0.5627468924937472, 'DT': 0.0, 'KNN': 1.5849265625000004}


* Are any of the models above underfitting the data? If so, which ones and how can you tell?
* Are any of thhe models above overfitting the data? If so, which ones and how can you tell?
* Which of the above models do you think fits the data best and why?

1.Yes. KNN is underfitting. Its training MSE is ≈ 1.585 and the test MSE is ≈ 2.149, and they are close. The model is too simple for the pattern.

2.Yes. Decision Tree is overfitting. Training MSE is 0, but test MSE is about ≈ 1.214, the big gap shows overfitting.

3.The Linear Regression fits best. The reason is that tt has the lowest test MSE, which is ≈ 0.813 and a small train–test gap.